# Парсинг отзывов с Яндекс.Маркета

Если посмотреть на отзывы в тесте, то можно понять, что нам нужны телефоны 2009-2013 года. К сожалению, даже если сегодня находятся покупатели похожих телефонов, их ожидания не соответствуют ожиданиям пользователей прошлых лет. Нам остается только надеяться, что в данных будут общие слова/словосочетания, по которым можно будет делать классификацию. На момент написания данного парсера, мною не были изучены более быстрые способы получения данных с Яндекс.Маркета. Поэтому если вы захотите использовать этот код для своих нужд, то будьте готовы подождать. Если Вам известны способы лучше, буду благодарен, если поделитесь.

In [2]:
import requests
import time
import bs4

import pandas as pd

from datetime import datetime

In [2]:
def df_maker(train_data, name):
    data1 = []
    data2 = []
    data3 = []

    for nov in train_data[0]:
        data1.append(nov[13:nov.find('Недостатки: ')].replace('\n', ' ')) #считываем из текста отзыва "Достоинства" #''

        if nov.find('Недостатки: ') != -1:  #считываем из текста отзыва "Недостатки"
            data2.append(nov[nov.find('Недостатки: ') + 12: nov.find('Комментарий: ')].replace('\n', ' '))
        else:
            data2.append('')

        if nov.find('Комментарий: ') != -1:  #считываем из текста отзыва "Комментарий"
            idx = nov.find('Комментарий: ') + 13
            data3.append(nov[idx: ].replace('\n', ' '))
        else:
            data3.append('')

    data4 = [nov for nov in train_data[1]]  #считываем оценки пользователей
    data5 = [nov for nov in train_data[2]]  #считываем оценки пользователей этого комметария 
    
    global df
    
    for i in range(len(data1)): #добавляем в DataFrame 
        df = df.append({'Достоинства': data1[i], 'Недостатки': data2[i], 'Комментарий': data3[i], 
                        'Рейтинг': data4[i], 'Оценка комментария': data5[i], 
                        'Название телефона': name }, ignore_index=True)

In [3]:
def data_miner(URL, name, i):
    
    comment_list = []
    appraisal_phone = []
    appraisal_comment = []

    URL += str(i)
    req = requests.get(URL)
    parser = bs4.BeautifulSoup(req.text, 'lxml')
        
    for nov in parser.findAll('div', attrs={'class':'_3IXczk7DdZ'}): #считывем тексты отзывов на этой странице
        comment_list.append(nov.text)
            
    for nov in parser.findAll('div', attrs={'class':'Ymh5FFxXk_ _2QBYNzUrMp autotest-RatingStars'}): #считываем оценки
        nov = str(nov)
        appraisal_phone.append(int(nov[nov.find('data-rate=') + 11]))
        
    for nov in parser.findAll('div', attrs={'class':'_2-SONC8w9Z'}): #считываем оценку отзыва на товар (лайк/дизлайк)
        tmp = nov.text.split()[-2:]
        tmp = [int(nov) for nov in tmp]
        try:
            tmp = float('{:.2f}'.format(tmp[0] / (tmp[0] + tmp[1])))
        except:
            tmp = -1 # так будем помечать отзывы у которых нет оценок другими пользователями
        appraisal_comment.append(tmp)
        
    print(i, '--', datetime.now().strftime("%H:%M:%S"), appraisal_phone, name)
    
    df_maker([comment_list, appraisal_phone, appraisal_comment], name)
    
    if i == 1: # считываем кол-во страниц с отзывами, на каждой странице не больше 10 отзывов
        n = int(parser.find('li', attrs={'class':'_1OM4gu7kXK QjE88eF2HX kSWkhB-y4k'}).text[6: ]) // 10 + 1
        if n > 20: # не будем считывать больше 20*10 отзывов на один телефон
            n = 20
        return n

In [11]:
# список названий телефонов
apparat_names = ['Samsung Galaxy A50 64GB', 'Apple iPhone Xr 64GB', 'Sony Xperia 5', 'Apple iPhone 7 32GB',
                 'Nokia 6300 4G', 'Nokia 8000 4G', 'Nokia 2720 Flip Dual sim', 'Philips Xenium E182', 
                 'Nokia 8110 4G', 'Nokia 230 Dual Sim']

# список соотвествующих ссылок
URLs =  ['https://market.yandex.ru/product--smartfon-samsung-galaxy-a50-64gb/394273081/reviews?show-'\
         'uid=16075284185418137152316180&track=srchlink&page=', 'https://market.yandex.ru/product--smartfon-'\
         'apple-iphone-xr-64gb/175941311/reviews?track=tabs&onstock=1&glfilter=16816262%3A16816264&page=', 
         'https://market.yandex.ru/product--smartfon-sony-xperia-5/553247010/reviews?track=tabs&page=', 
         'https://market.yandex.ru/product--smartfon-apple-iphone-7-32gb/14206636/reviews?track=tabs&onstock=1&'\
         'glfilter=16816262%3A16816264&page=','https://market.yandex.ru/product--telefon-nokia-6300-4g/755200002/reviews'\
         '?track=tabs&cpa=0', 'https://market.yandex.ru/product--telefon-nokia-8000-4g/755201000/reviews?track=tabs&'\
         'cpa=0', 'https://market.yandex.ru/product--telefon-nokia-2720-flip-dual-sim/584488006/reviews?track='\
         'tabs&onstock=1&glfilter=16816262%3A16816263&page=', 'https://market.yandex.ru/product--telefon-philips-'\
         'xenium-e182/463953099/reviews?track=tabs&onstock=1&glfilter=16816262%3A16816263&page=', 
         'https://market.yandex.ru/product--telefon-nokia-8110-4g/1969020910/reviews?track=tabs&onstock=1&glfilter='\
         '16816262%3A16816263&page=', 'https://market.yandex.ru/product--telefon-nokia-230-dual-sim/13139098/reviews?'\
         'track=tabs&onstock=1&glfilter=16816262%3A16816263&page=']


try:
    df = pd.read_csv('train_data.csv')
except:
    df = pd.DataFrame({'Достоинства': [], 'Недостатки': [], 'Комментарий': [], 
                       'Рейтинг': [], 'Оценка комментария': [], 'Название телефона': []})
flag = True
for i, (URL, apparat_name) in enumerate(zip(URLs, apparat_names)):
    if flag:
        n = data_miner(URL, apparat_name, 1)
        df.to_csv('train_data.csv', index=False)
        flag = False
        
    for j in range(2, n + 1):
        data_miner(URL, apparat_name, j)
        df.to_csv('train_data.csv', index=False)

        if j % 2 == 0 and j != n:
            time.sleep(4000)

    flag = True
    if i != len(URLs) - 1:
        time.sleep(4000)